In [2]:
import json
import os
import pandas as pd

from dotenv import load_dotenv
from typing import List, Dict, Any
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [3]:
pd.set_option('display.width', -1)
pd.set_option('max_colwidth', 1000)

In [4]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("Issue detected with API key")
    
openai = OpenAI()

API key looks good so far


In [5]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

### 1. Load JSON data
---

In [6]:
# Load JSON data
with open('./data/convfinqatrain.json', 'r') as f:
    json_data = json.load(f)
data = json_data[:5]

### 2. Helper Functions for processing financial documents encoded as JSON file
---

In [7]:
def table_to_string(table):
    """Convert table to string representation."""
    return "\n".join(" | ".join(str(cell) for cell in row) for row in table)

In [8]:
def process_json_entry(entry: Dict[str, Any]) -> Dict[str, Any]:
    """Process a single JSON entry to extract id, merged text, and table string."""
    entry_id = entry.get("id", None)
    parts = []

    # Merge pre_text if available
    if "pre_text" in entry and entry["pre_text"]:
        parts.append(" ".join(entry["pre_text"]))

    # Convert table to a string if it exists
    table_str = table_to_string(entry["table"]) if "table" in entry and entry["table"] else ""
    if table_str:
        parts.append(table_str)

    # Merge post_text if available
    if "post_text" in entry and entry["post_text"]:
        parts.append(" ".join(entry["post_text"]))

    # Merge QA details
    if "qa" in entry and entry["qa"]:
        qa_parts = []
        if "question" in entry["qa"]:
            qa_parts.append(f"Question: {entry['qa']['question']}")
        if "answer" in entry["qa"]:
            qa_parts.append(f"Answer: {entry['qa']['answer']}")
        if "exe_ans" in entry["qa"]:
            qa_parts.append(f"Execution Answer: {entry['qa']['exe_ans']}")
        parts.append("\n".join(qa_parts))
    
    # Extract and Merge dialogue break
    dialogue_break = entry.get("annotation", {}).get("dialogue_break", "")
    if dialogue_break:
        parts.append(f"Dialogue Break: {dialogue_break}")

    # Combine all parts into a single text field
    full_text = "\n\n".join(parts)

    return {"id": entry_id, "text": full_text, "possible_questions":dialogue_break, "table": table_str}    

In [9]:
def get_table_description(table_content: str, document_context: str) -> str:
    """
    Given the table content and the document context, use OpenAI's ChatCompletion API
    to generate a detailed description of the table and include the table in markdown format.
    """
    prompt = f"""
    Given the following table and its context from the original document,
    provide a detailed description of the table. Then, include the table in markdown format.

    Original Document Context:
    {document_context}

    Table Content:
    {table_content}

    Please provide:
    1. A comprehensive description of the table.
    2. The table in markdown format.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that describes tables and formats them in markdown."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

### 3. Process each JSON entry
---

In [10]:
processed_data = [process_json_entry(entry) for entry in data]

In [11]:
df = pd.DataFrame(processed_data)
df

,id,text,possible_questions,table
0,Single_JKHY/2009/page_28.pdf-3,"26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 . all revenue components within the segment experienced growth during fiscal 2008 . license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year . support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support . gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins . liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations and short-term borrowings on our re...","[what is the net cash from operating activities in 2009?, what about in 2008?, what is the difference?, what percentage change does this represent?]",2008 | year ended june 30 2009 2008 | year ended june 30 2009 2008 | year ended june 30 2009\nnet income | $ 103102 | $ 104222 | $ 104681\nnon-cash expenses | 74397 | 70420 | 56348\nchange in receivables | 21214 | -2913 ( 2913 ) | -28853 ( 28853 )\nchange in deferred revenue | 21943 | 5100 | 24576\nchange in other assets and liabilities | -14068 ( 14068 ) | 4172 | 17495\nnet cash from operating activities | $ 206588 | $ 181001 | $ 174247
1,Single_RSG/2008/page_114.pdf-2,"substantially all of the goodwill and other intangible assets recorded related to the acquisition of allied are not deductible for tax purposes . pro forma information the consolidated financial statements presented for republic include the operating results of allied from the date of the acquisition . the following pro forma information is presented assuming the merger had been completed as of january 1 , 2007 . the unaudited pro forma information presented below has been prepared for illustrative purposes and is not intended to be indicative of the results of operations that would have actually occurred had the acquisition been consummated at the beginning of the periods presented or of future results of the combined operations ( in millions , except share and per share amounts ) . year ended december 31 , year ended december 31 , ( unaudited ) ( unaudited ) .\n\n | year ended december 31 2008 ( unaudited ) | year ended december 31 2007 ( unaudited )\nrevenue | $ 9362.2 | $ 9244....","[what were revenues in 2008?, what were they in 2007?, what was the net change?, what is the percent change?]",| year ended december 31 2008 ( unaudited ) | year ended december 31 2007 ( unaudited )\nrevenue | $ 9362.2 | $ 9244.9\nincome from continuing operations available to common stockholders | 285.7 | 423.2\nbasic earnings per share | .76 | 1.10\ndiluted earnings per share | .75 | 1.09
2,Single_AAPL/2002/page_23.pdf-1,"in a new business model such as the retail segment is inherently risky , particularly in light of the significant investment involved , the current economic climate , and the fixed nature of a substantial portion of the retail segment's operating expenses . results for this segment are dependent upon a number of risks and uncertainties , some of which are discussed below under the heading ""factors that may affect future results and financial condition."" backlog in the company's experience , the actual amount of product backlog at any particular time is not a meaningful indication of its future business prospects . in particular , backlog often increases in anticipation of or immediately following new product introductions because of over- ordering by dealers anticipating shortages . backlog often is reduced once dealers and customers believe they can obtain su

### 4. Generate contextualised table description
---

In [12]:
def safe_get_table_description(row):
    table_content = row["table"]
    document_context = row["text"]
    if table_content and table_content.strip():
        try:
            return get_table_description(table_content, document_context)
        except Exception as e:
            print(f"Error processing table description for id {row['id']}: {e}")
            return ""
    else:
        return ""

In [13]:
df["table_description"] = df.apply(safe_get_table_description, axis=1)

In [14]:
df

,id,text,possible_questions,table,table_description
0,Single_JKHY/2009/page_28.pdf-3,"26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 . all revenue components within the segment experienced growth during fiscal 2008 . license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year . support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support . gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins . liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations and short-term borrowings on our re...","[what is the net cash from operating activities in 2009?, what about in 2008?, what is the difference?, what percentage change does this represent?]",2008 | year ended june 30 2009 2008 | year ended june 30 2009 2008 | year ended june 30 2009\nnet income | $ 103102 | $ 104222 | $ 104681\nnon-cash expenses | 74397 | 70420 | 56348\nchange in receivables | 21214 | -2913 ( 2913 ) | -28853 ( 28853 )\nchange in deferred revenue | 21943 | 5100 | 24576\nchange in other assets and liabilities | -14068 ( 14068 ) | 4172 | 17495\nnet cash from operating activities | $ 206588 | $ 181001 | $ 174247,"### Description of the Table\n\nThe table summarizes the net cash from operating activities for a company over the fiscal years ending June 30 from 2007 to 2009. It includes detailed line items that contribute to the calculation of net cash from operating activities, which are crucial for understanding the company's financial performance and cash flow situation.\n\n1. **Lines Item Breakdown:**\n - **Net Income:** This row shows the net income for each of the three fiscal years. It indicates a slight decrease in net income from 2007 to 2009.\n - **Non-Cash Expenses:** This represents various non-cash charges such as depreciation and amortization, which are added back to net income to determine cash flows.\n - **Change in Receivables:** This indicates the net change in accounts receivable, showing how much cash was collected as compared to how much was billed.\n - **Change in Deferred Revenue:** This row reflects changes in deferred revenue, which contributes to cash inflows ..."
1,Single_RSG/2008/page_114.pdf-2,"substantially all of the goodwill and other intangible assets recorded related to the acquisition of allied are not deductible for tax purposes . pro forma information the consolidated financial statements presented for republic include the operating results of allied from the date of the acquisition . the following pro forma information is presented assuming the merger had been completed as of january 1 , 2007 . the unaudited pro forma information presented below has been prepared for illustrative purposes and is not intended to be indicative of the results of operations that would have actually occurred had the acquisition been consummated at the beginning of the periods presented or of future results of the combined operations ( in millions , except share and per share amounts ) . year ended december 31 , year ended december 31 , ( unaudited ) ( unaudited ) .\n\n | year ended december 31 2008 ( unaudited ) | year ended december 31 2007 ( unaudited )\nrevenue | $ 9362.2 | $ 9244....","[what were revenues in 2008?, what were they in 2007?, what was the net change?, what is the percent change?]",| year ended december 31 2008 ( unaudited ) | year ended december 31 2007 ( unaudited )\nrevenue | $ 9362.2 | $ 9244.9\nincome from continuing operations available to common stock

### 5. Append contextualised table field to text field
---

In [15]:
df["expanded_text"] = df["text"] + "\n\n" + df["table_description"]

### 6. Run embedding model
---

In [ ]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
df["embedding"] = df["expanded_text"].apply(lambda x: embedding_model.embed_query(x) if x else [])
df

/var/folders/4f/c9ycdnn56lx6fjnv2my0qwk80000gn/T/ipykernel_4263/1236624881.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")


,id,text,possible_questions,table,table_description,expanded_text,embedding
0,Single_JKHY/2009/page_28.pdf-3,"26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 . all revenue components within the segment experienced growth during fiscal 2008 . license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year . support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support . gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins . liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations and short-term borrowings on our re...","[what is the net cash from operating activities in 2009?, what about in 2008?, what is the difference?, what percentage change does this represent?]",2008 | year ended june 30 2009 2008 | year ended june 30 2009 2008 | year ended june 30 2009\nnet income | $ 103102 | $ 104222 | $ 104681\nnon-cash expenses | 74397 | 70420 | 56348\nchange in receivables | 21214 | -2913 ( 2913 ) | -28853 ( 28853 )\nchange in deferred revenue | 21943 | 5100 | 24576\nchange in other assets and liabilities | -14068 ( 14068 ) | 4172 | 17495\nnet cash from operating activities | $ 206588 | $ 181001 | $ 174247,"### Description of the Table\n\nThe table summarizes the net cash from operating activities for a company over the fiscal years ending June 30 from 2007 to 2009. It includes detailed line items that contribute to the calculation of net cash from operating activities, which are crucial for understanding the company's financial performance and cash flow situation.\n\n1. **Lines Item Breakdown:**\n - **Net Income:** This row shows the net income for each of the three fiscal years. It indicates a slight decrease in net income from 2007 to 2009.\n - **Non-Cash Expenses:** This represents various non-cash charges such as depreciation and amortization, which are added back to net income to determine cash flows.\n - **Change in Receivables:** This indicates the net change in accounts receivable, showing how much cash was collected as compared to how much was billed.\n - **Change in Deferred Revenue:** This row reflects changes in deferred revenue, which contributes to cash inflows ...","26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 . all revenue components within the segment experienced growth during fiscal 2008 . license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year . support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support . gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins . liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations and short-term borrowings on our re...","[-0.04168519960464352, 0.025579553964186694, 0.05294714289388926, 0.021242906443692415, 0.052226589821835034, -0.01531837634704506, -0.02178999137975278, 0.01758677816461867, 0.010514704101177505, 0.01606561367246821, 0.00743234827116193, -0.0066917820090810105, -0.023177717841252915, -0.0313306177

### 7. Create ChromdaDB vector store of documents
---

In [17]:
# Create documents for vector store
documents = []

In [18]:
documents = [
    Document(
        page_content=row["expanded_text"],
        metadata={
            "id": row["id"],
            "table": row["table"],
            "table_description": row["table_description"]
        }
    )
    for _, row in df.iterrows()
]

In [19]:
documents

[Document(metadata={'id': 'Single_JKHY/2009/page_28.pdf-3', 'table': '2008 | year ended june 30 2009 2008 | year ended june 30 2009 2008 | year ended june 30 2009\nnet income | $ 103102 | $ 104222 | $ 104681\nnon-cash expenses | 74397 | 70420 | 56348\nchange in receivables | 21214 | -2913 ( 2913 ) | -28853 ( 28853 )\nchange in deferred revenue | 21943 | 5100 | 24576\nchange in other assets and liabilities | -14068 ( 14068 ) | 4172 | 17495\nnet cash from operating activities | $ 206588 | $ 181001 | $ 174247', 'table_description': "### Description of the Table\n\nThe table summarizes the net cash from operating activities for a company over the fiscal years ending June 30 from 2007 to 2009. It includes detailed line items that contribute to the calculation of net cash from operating activities, which are crucial for understanding the company's financial performance and cash flow situation.\n\n1. **Lines Item Breakdown:**\n   - **Net Income:** This row shows the net income for each of the

In [20]:
# Initialise ChromaDB vector store
persist_directory = "vectorstore"

In [21]:
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory=persist_directory
)
vectorstore.persist()

/var/folders/4f/c9ycdnn56lx6fjnv2my0qwk80000gn/T/ipykernel_4263/299267684.py:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


### 8. Create the Prompt Template
---

In [22]:
FIN_PROMPT_TEMPLATE = """
You are a financial analyst assistant. Use the following pieces of context to answer the question about financial data in tables. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}

Question: {question}

Provide a detailed answer with numerical calculations when applicable:
"""

In [23]:
PROMPT = PromptTemplate(
    template=FIN_PROMPT_TEMPLATE,
    input_variables=["context", "question"]
)

### 9. Query the vector store
---

In [24]:
# Initialize QA chain
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True
)

/var/folders/4f/c9ycdnn56lx6fjnv2my0qwk80000gn/T/ipykernel_4263/668712054.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [25]:
def query_financial_data(question):
    """
    Query the financial data using the QA chain.
    
    Args:
        question (str): The question to ask about the financial data
        
    Returns:
        Dict: Contains the answer and source documents
    """
    result = qa_chain({"query": question})
    return {
        "answer": result["result"],
        "source_documents": [doc.page_content for doc in result["source_documents"]]
    }

In [32]:
question = "What was the net cash from operating activities in 2008?"

In [33]:
result = query_financial_data(question)

In [34]:
print(result["answer"])

The net cash from operating activities in 2008 was $181,001. 

This figure is taken directly from the provided table summarizing net cash from operating activities for the fiscal years ending June 30 from 2007 to 2009. Here is the relevant portion of the table for clarity:

```markdown
| Line Item                           | 2009         | 2008         | 2007         |
|-------------------------------------|--------------|--------------|--------------|
| **Net Cash from Operating Activities** | **$ 206,588** | **$ 181,001** | **$ 174,247** |
```

Thus, the net cash from operating activities for the fiscal year ended June 30, 2008, was **$181,001**.
